In [2]:
import pandas as pd

from pydantic import BaseModel
from typing import List, Optional

from backend.classes.AttendanceValues import AttendanceValues
from backend.classes.StudentMetrics import StudentMetrics
from backend.classes.StudentTrend import StudentTrend

In [ ]:
df = pd.read_parquet('backend/data/Alerts.parquet')

NameError: name 'pd' is not defined

In [3]:
class DataResponse(BaseModel):
    previousAttendance: float | None
    predictedAttendance: float
    predictedValues: AttendanceValues
    chronicAbsenceRate: float
    metrics: List[StudentMetrics]
    trends: List[StudentTrend]

In [5]:
def _subset_pairs(data: pd.DataFrame):
    cur, pred = 2024, 2025
    hist = data[data["SCHOOL_YEAR"] <= cur]
    pr   = data[data["SCHOOL_YEAR"] == pred]
    return hist, pr

In [32]:
PRESENT_COL  = "Total_Days_Present"
ENROLLED_COL  = "Total_Days_Enrolled"
UNEXCUSED_COL = "Total_Days_Unexcused_Absent"
PRED_COL = "Predictions"
PRED_DIST_COL = "Predictions_District"
PRED_SCH_COL  = "Predictions_School"
PRED_GRD_COL  = "Predictions_Grade"

def get_all_districts_summary():
    hist, pred = _subset_pairs(df)
    cur_year = 2024
    cur_rows = hist[hist["SCHOOL_YEAR"] == cur_year]

    present_tot  = cur_rows['Total_Days_Present'].astype(float).sum()
    enrolled_tot = cur_rows['Total_Days_Enrolled'].astype(float).sum()
    prev_att = round((present_tot / enrolled_tot) * 100, 1) if enrolled_tot > 0 else 0
    total_days = round(enrolled_tot / len(cur_rows), 1)
    preds = pred[PRED_DIST_COL].dropna()
    pred_att = round(preds.mean() * 100, 1) if not preds.empty else 0
    chronic_absence_rate = round(((preds[preds < 0.85].shape[0]) / cur_rows.shape[0]) * 100, 2)
    

    return_vals = {
        'previousAttendance': prev_att,
        'predictedAttendance': pred_att,
        'chronicAbsenceRate': chronic_absence_rate
    }

    return return_vals

In [33]:
get_all_districts_summary()

{'previousAttendance': np.float64(90.1),
 'predictedAttendance': np.float64(89.0),
 'chronicAbsenceRate': 4.12}